$$
Modelo
$$

In [4]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import gradio as gr

In [5]:
# ingesta de datos
df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id','title'])
df_score = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId','score','movieId'])

In [6]:
df_movie.head()

,id,title
0,ns1,dick johnson is dead
1,ns2,blood & water
2,ns3,ganglands
3,ns4,jailbirds new orleans
4,ns5,kota factory


In [7]:
df_score.head()

,userId,score,movieId
0,1,1.0,as680
1,1,4.5,ns2186
2,1,5.0,hs2381
3,1,5.0,ns3663
4,1,5.0,as9500


visualizar solo las columnas que usare para el modelo y la recomendacion

In [8]:
df_score[df_score['userId']==1].sort_values('score', ascending=False)

,userId,score,movieId
26,1,5.0,ns1310
10,1,5.0,ns7360
21,1,5.0,hs871
19,1,5.0,as4340
18,1,5.0,as6909
17,1,5.0,ns5413
16,1,5.0,hs2056
15,1,5.0,as1577
23,1,5.0,ns8584
11,1,5.0,ns3980


En este filtro se puede observar las puntuaciones del usuario 1. que le otorgo a un grupo de peliculas la puntuacion de 5, esos reflejan mas sus intereses y por medio de K-Vecinos mas cercanos puedo calcular que recomendar a otro usuarios que tengan intereses similares 

In [9]:
#Definir la funcion Obtener Titulos
def obtener_titulo(userId, score):
    df_filtrado = df_score[(df_score['userId']==userId) & (df_score['score']==score)]
    merged_df = pd.merge(df_filtrado, df_movie, left_on='movieId', right_on='id', how='inner')
    titulos = merged_df['title'].to_list()
    return '\n'.join(titulos)
#crear una instancia de usario con Gradio
iface = gr.Interface(
    fn=obtener_titulo,
    inputs=["number","number"],
    outputs="text",
    title="Listar Peliculas por Usuario y Puntaje",
    description="Ingrese un userId y ina puntuacion para obtener los titulos correspondiente"
    )
# Ejecutar la interface de usuario
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Construccion del modelo

In [10]:
#importar librerias
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader, SVD

In [11]:
#importar datos
df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id','title'])
df_movie = df_movie.reset_index(drop=True)
df_score = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId','score','movieId'])


In [12]:
#crear df filtrado con los puntajes que solo sean 5
df_filtered = df_score[df_score['score'] == 5]

In [13]:
df_filtered.head()

,userId,score,movieId
2,1,5.0,hs2381
3,1,5.0,ns3663
4,1,5.0,as9500
7,1,5.0,as6112
10,1,5.0,ns7360


In [14]:
#verifico que no contenga otra puntuacion en el df filtrado
df_filtered[df_filtered['score']!=5]

,userId,score,movieId


In [15]:
#verificar que no existan valores nulos en el df
df_filtered.isna().sum()

userId     0
score      0
movieId    0
dtype: int64

In [16]:
df_movie.head()

,id,title
0,ns1,dick johnson is dead
1,ns2,blood & water
2,ns3,ganglands
3,ns4,jailbirds new orleans
4,ns5,kota factory


In [17]:
#verificar nulos en df de peliculas
df_movie.isna().sum()

id       0
title    0
dtype: int64